In [ ]:
from google.colab import drive
import os
import sys

drive.mount("/content/drive")

# install dependencies:
!pip install pyyaml==5.1
!pip install torch==1.8.0 torchvision==0.9.0

import torch
import torchvision

print(torch.__version__, torch.cuda.is_available())
!gcc --version
# install detectron2: (colab has CUDA 10.1 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
assert torch.__version__.startswith("1.8")
!pip install detectron2==0.4 -q -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# Colab's "restart runtime" is needed after installation
!pip install -q -U albumentations

os.chdir("drive/MyDrive/PG/pg_iss/")
assert os.getcwd().split("/")[-1] == "pg_iss"
# clone the repo at the same version, in order to access pre-defined configs
# !git clone --branch v0.4 https://github.com/facebookresearch/detectron2.git detectron2_repo
!pip install -q -e detectron2_repo/projects/TensorMask/

In [ ]:
%load_ext autoreload
%autoreload 2

from detectron2.utils import comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.engine import (
    DefaultTrainer,
    DefaultPredictor,
    default_argument_parser,
    default_setup,
    launch,
)
from detectron2.evaluation import COCOEvaluator, verify_results
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
import random, cv2
import numpy as np
from google.colab.patches import cv2_imshow

sys.path.append("detectron2_repo/projects/TensorMask")

from lib import coco_handler, copy_and_paste_augm, constants
from lib.copy_and_paste_detectron import *

from tensormask import add_tensormask_config
import tensormask

import warnings

warnings.filterwarnings("ignore", "(Possibly )?corrupt EXIF data", UserWarning)
os.environ["LRU_CACHE_CAPACITY"] = "1"

In [ ]:
cfg = get_cfg()
add_tensormask_config(cfg)
add_cap_config(cfg)
cfg.merge_from_file("configs/CAP_config.yaml")
cfg.OUTPUT_DIR = "./output/run/"
# cfg.OUTPUT_DIR = "/content/tmp/output/"  # use tmp for testing, change to drive
# cfg.CAP.PATCH_POOL = "data/copy_and_paste/objs-test/"  # use small dummy pool
# cfg.CAP.PICKLE_PATH = "/content/tmp/pickle/cap_dataset.pkl"
create_output(cfg)

In [ ]:
dataset = CapDataset(cfg)
dataset.dump(cfg=cfg)
# dataset.dump(cfg=cfg, path="/content/tmp/pickle/cap_dataset.pkl")  # use tmp storage
n_examples = 5
for i in range(n_examples):  # show some examples
    CapDataset.show_image_and_mask(dataset[i])
if n_examples:
    dataset.visualize_pools()

In [ ]:
register_coco_instances(
    "boxes", {}, "data/raw/annotations/all_anno.json", "data/raw/imgs"
)
cfg.DATASETS.TEST = ("boxes",)
dataset_dicts = DatasetCatalog.get("boxes")

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./output/

In [ ]:
trainer = Trainer(cfg)  # might take some time for creating object pools
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
from detectron2.utils.visualizer import ColorMode, Visualizer
import random, cv2, glob

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
predictor = DefaultPredictor(cfg)

for path in glob.glob(os.path.join(constants.path_to_data_dir, "raw", "imgs/*")):
    im = cv2.imread(path)
    outputs = predictor(im)
    v = Visualizer(
        im[:, :, ::-1],
        metadata=MetadataCatalog.get("boxes"),
        scale=0.8,
        instance_mode=ColorMode.IMAGE_BW,  # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])